In [2]:
# ============================================================================
# SETUP: Import LLM Helper Functions
# ============================================================================
# We use helper functions to create LLM instances with proper configuration
# These functions handle API key loading and model configuration

import os
import sys

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference)
# - get_openai_llm(): Creates an OpenAI GPT model
from helpers.utils import get_groq_llm, get_openai_llm,get_databricks_llm

print("LLM helpers imported successfully!")

# ============================================================================
# CREATE THE LLM AND CHATBOT GRAPH
# ============================================================================

# -----------------------------------------------------------------------------
# Step 1: Initialize the LLM
# We use Groq for fast inference, but you can swap to OpenAI
# -----------------------------------------------------------------------------
llm = get_databricks_llm("databricks-gemini-2-5-pro")  # Fast, open-source models hosted by Groq
# Alternative: llm = get_openai_llm()  # OpenAI's GPT models

if hasattr(llm, 'model_name'):
    print(f"LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"LLM initialized: {llm.model} (Databricks)")
else:
    print("LLM initialized: Groq LLM")

LLM helpers imported successfully!
LLM initialized: databricks-gemini-2-5-pro (Databricks)


## Basic Tool Calling Example

In [3]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Define custom tools
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city."""
    # Simulated weather data
    weather_data = {
        "bangalore": "28°C, Partly Cloudy",
        "mumbai": "32°C, Humid",
        "delhi": "25°C, Sunny"
    }
    return weather_data.get(city.lower(), "Weather data not available")

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Bind tools to the LLM
tools = [get_weather, calculate]
llm_with_tools = llm.bind_tools(tools)

# Invoke with a query
response = llm_with_tools.invoke("What's the weather in Bangalore?")
print(response.tool_calls)


[{'name': 'get_weather', 'args': {'city': 'Bangalore'}, 'id': 'get_weather', 'type': 'tool_call'}]


## Complete Agent Example with Tool Execution

In [ ]:
from langchain_core.tools import tool
from langchain.agents import create_agent

# Define tools
@tool
def search_database(query: str) -> str:
    """Search the database for information about a topic."""
    return f"Found 3 results for: {query}"

@tool
def get_stock_price(symbol: str) -> str:
    """Get the current stock price for a given symbol."""
    prices = {"GOOGL": 175.50, "MSFT": 420.30, "AAPL": 195.80}
    price = prices.get(symbol.upper(), "Not found")
    return f"{symbol.upper()}: ${price}"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to the specified recipient."""
    return f"Email sent to {to} with subject: {subject}"

tools = [search_database, get_stock_price, send_email]

# Create agent using the new LangChain 1.x API
# create_agent returns a compiled graph that can be invoked directly
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant. Use tools when needed."
)

# Run the agent
# The new API uses "messages" as input instead of "input"
result = agent.invoke({
    "messages": [{"role": "user", "content": "What's the stock price of Google?"}]
})

# Extract the final response
final_message = result["messages"][-1]



[{"type": "text", "text": "I can get you the stock price, but I need to know which stock symbol you're interested in (for example, GOOGL).", "thoughtSignature": "CvQIAY89a1/IIO9cOWUxPzqtCyINA5JSu+cEOJIAClPGxpUpkaAtIoTwR3HVyvnhwDd/fnAIjQvPu5hioIbBUSK9Jd40LtRrUmpSGIrd6MRXosDTSasoP7z9Ybs5rEKPOOVLvBjcnUsslamx7Q+JiIRqlhIlfGMgnG1HkyzHHc0LCxSwP0bk2Fk7s5YypYy7zBU7w856yf5jgXaLOB2JyrvKhezjI+iECVMH7ZQVfWXU02kjw5KmBQpFVo1XJPIFPx7/oHM18iH3N58bz89s0nR/iTL3cxuXX5reVZ2nJAzNoiMMN9KrY5rOBTkUUmZLdDW11mebq+eJiFuQNnuOFLxgR7Hum8mhqFD70OdiUc9sN2kMOiQlbgvejQNbpaUR3jZBCGKilBJYGzrUP4YVJgzxQKZgs8DwOjSV5HcBA1szx3py/ERqqyychOkhWMTkM4RmVOJzGs1AXuT2foDbSZMqg7Nzq5147nNdW440DMeJZcbeIkyTwj1HG4yQQjZOlc51hTJvq/Ats92f5To7wK5SdsbQX1yr2PI9KLOnEIwPTjAD4P+IgOgDM8ctcz8e38+kX1crTljLauhrfZhVMRlyOSc253JDu4IKFITfg/WRQ1GIZv9fAqCmeUZILHtkE9OMiNLVDvwa0lI3OqkhZIhrjHKHWvgax7/Jrk7G+AMcmcLWB74krfL3bhD49f4PpROyhlFmKsGYs8MRNr0yBKMPcVD2ndYB4JErEJmBNqM1RoQLT54y4SDRpuEx/gkiaW0lWMvVZW7q0++eq4NAgblbKAhY8dZeKp6My3SYqXFDDjMnJCNcOIt0rw5BhNYiDVm0b1

/Users/sourav.banerjee/Documents/Codebases/2. AI ENGINEERING/LangChain_Demystified/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'text', 'text':...9x13KrJXoaGjzY/7+QIkH'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


In [7]:
print(final_message.content)

[{"type": "text", "text": "I can get you the stock price, but I need to know which stock symbol you're interested in (for example, GOOGL).", "thoughtSignature": "CvQIAY89a1/IIO9cOWUxPzqtCyINA5JSu+cEOJIAClPGxpUpkaAtIoTwR3HVyvnhwDd/fnAIjQvPu5hioIbBUSK9Jd40LtRrUmpSGIrd6MRXosDTSasoP7z9Ybs5rEKPOOVLvBjcnUsslamx7Q+JiIRqlhIlfGMgnG1HkyzHHc0LCxSwP0bk2Fk7s5YypYy7zBU7w856yf5jgXaLOB2JyrvKhezjI+iECVMH7ZQVfWXU02kjw5KmBQpFVo1XJPIFPx7/oHM18iH3N58bz89s0nR/iTL3cxuXX5reVZ2nJAzNoiMMN9KrY5rOBTkUUmZLdDW11mebq+eJiFuQNnuOFLxgR7Hum8mhqFD70OdiUc9sN2kMOiQlbgvejQNbpaUR3jZBCGKilBJYGzrUP4YVJgzxQKZgs8DwOjSV5HcBA1szx3py/ERqqyychOkhWMTkM4RmVOJzGs1AXuT2foDbSZMqg7Nzq5147nNdW440DMeJZcbeIkyTwj1HG4yQQjZOlc51hTJvq/Ats92f5To7wK5SdsbQX1yr2PI9KLOnEIwPTjAD4P+IgOgDM8ctcz8e38+kX1crTljLauhrfZhVMRlyOSc253JDu4IKFITfg/WRQ1GIZv9fAqCmeUZILHtkE9OMiNLVDvwa0lI3OqkhZIhrjHKHWvgax7/Jrk7G+AMcmcLWB74krfL3bhD49f4PpROyhlFmKsGYs8MRNr0yBKMPcVD2ndYB4JErEJmBNqM1RoQLT54y4SDRpuEx/gkiaW0lWMvVZW7q0++eq4NAgblbKAhY8dZeKp6My3SYqXFDDjMnJCNcOIt0rw5BhNYiDVm0b1